In [21]:
import duckdb
import os.path

FLUTTER_PARQUET = os.path.expanduser("~/src/ripe/rpki/rpki-flutter/data/*.parquet")

In [22]:
tbl = duckdb.read_parquet(FLUTTER_PARQUET)

In [9]:
def prefix_first_to_bytes(inp: str) -> bytes:
    range = netaddr.IPNetwork(inp)
    buf = ipaddress.ip_address(range.first).packed
    to_pad = len(buf) - (128//8)

    return b'\x00' * to_pad + buf

def prefix_last_to_bytes(inp: str) -> bytes:
    range = netaddr.IPNetwork(inp)
    buf = ipaddress.ip_address(range.last).packed
    to_pad = len(buf) - (128//8)

    return b'\x00' * to_pad + buf

duckdb.create_function("prefix_first", prefix_first_to_bytes)
duckdb.create_function("prefix_last", prefix_last_to_bytes)


In [23]:
duckdb.query("select prefix_first(pfx) from (select * from '" + FLUTTER_PARQUET + "' where pfx = '1.0.0.0/24')")

┌───────────────────┐
│ prefix_first(pfx) │
│       blob        │
├───────────────────┤
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│        ·          │
│        ·          │
│        ·          │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
│ \x01\x00\x00\x00  │
├───────────────────┤
│     124 rows      │
│    (20 shown)     │
└───────────────────┘